In [1]:
import pandas as pd

In [2]:
#data1 = pd.read_csv('/content/drive/MyDrive/CAPSTONE WORK/chunked_data/data_male_female1.csv')
#data2 = pd.read_csv('/content/drive/MyDrive/CAPSTONE WORK/chunked_data/data_male_female2.csv')
#data3 = pd.read_csv('/content/drive/MyDrive/CAPSTONE WORK/chunked_data/data_male_female3.csv')
#data4 = pd.read_csv('/content/drive/MyDrive/CAPSTONE WORK/chunked_data/data_male_female4.csv')
#data = pd.read_csv('/content/drive/MyDrive/CAPSTONE WORK/chunked_data/data_male_female5.csv')

In [3]:
data = pd.read_csv('/content/drive/MyDrive/CAPSTONE WORK/chunked_data/data_male_female2.csv')

FileNotFoundError: ignored

In [ ]:
data.shape

In [ ]:
data.head(1)

Topic Modeling Code

In [ ]:
# imports
import re
import numpy as np
from pprint import pprint
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
# spacy for lemmatization
import spacy
import nltk
nltk.download('wordnet')
nltk.download('words')
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

import matplotlib.pyplot as plt

# Enable logging for gensim - optional
import logging
logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
# preprocessing

# Convert to list
title_list = data['title:string'].values.tolist()

title_list = [re.sub('\S*@\S*\s?', '', str(sent)) for sent in title_list]

# Remove new line characters
title_list = [re.sub('\s+', ' ', sent) for sent in title_list]

# Remove distracting single quotes
title_list = [re.sub("\'", "", sent) for sent in title_list]

print(title_list[:1])

In [ ]:
title_list = [re.sub("-", " ", sent) for sent in title_list]
title_list = [re.sub(":", "", sent) for sent in title_list]

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

title_words_list = list(sent_to_words(title_list))

title_words_list[:1]

bigram trigram

In [ ]:
# Build the bigram and trigram models

bigram = gensim.models.Phrases(title_words_list, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[title_words_list], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

and lemmatize

In [ ]:
from gensim.utils import simple_preprocess
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use','a','about', 'above', 'across'])

In [ ]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [ ]:
# Remove Stop Words

title_words_nostops = remove_stopwords(title_words_list)

# Form Bigrams

title_words_bigrams = make_bigrams(title_words_nostops)

titles_lemmatized = []

for title in title_words_bigrams:
    new_title = []
    for word in title:
        new_title.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
    titles_lemmatized.append(new_title)

titles_lemmatized = [' '.join(i) for i in titles_lemmatized]

titles_lemmatized = [titles.split() for titles in titles_lemmatized]

titles_lemmatized

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(titles_lemmatized)

# Create Corpus
texts = titles_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[11:12]]

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=9, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=1000,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
# Compute Perplexity

print('\nPerplexity: ', lda_model.log_perplexity(corpus)) 

# Compute Coherence Score

coherence_model_lda = CoherenceModel(model=lda_model, texts=titles_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# evaluating for optimal coherence score
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = LdaModel(corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, 
                                                        texts=titles_lemmatized, start=1, limit=15, step=1)

In [ ]:
limit=15; start=1; step=1;
x = range(start, limit, step)
plt.figure(figsize=(12,12))
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=1000,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
   
    sent_topics_df = pd.DataFrame()

   
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=False)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # -- dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)
    

df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=title_list)


df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']


df_dominant_topic.head(5)

In [ ]:
df_dominant_topic.shape

In [ ]:
 all_topics = lda_model.get_document_topics(corpus, minimum_probability=0.0)
 all_topics_csr = gensim.matutils.corpus2csc(all_topics)
 all_topics_numpy = all_topics_csr.T.toarray()
 all_topics_df = pd.DataFrame(all_topics_numpy)

In [ ]:
all_topics_df

In [ ]:
chunked_topics = pd.concat([data, df_dominant_topic], axis=1)

In [ ]:
chunked_topics

In [ ]:
chunked_topics.to_csv('/content/drive/MyDrive/CAPSTONE WORK/Topic Modeling Chunking/t7c.csv')